# データ拡張（Augmentation）自動生成スクリプト

In [ ]:
import os
import cv2
import albumentations as A
from tqdm import tqdm

# --- フォルダ設定 ---
IMG_DIR = "./data_train/v3/images/train"
LABEL_DIR = "./data_train/v3/labels/train"
OUT_IMG_DIR = "./data_train/v3/images/train"
OUT_LABEL_DIR = "./data_train/v3/labels/train"

os.makedirs(OUT_IMG_DIR, exist_ok=True)
os.makedirs(OUT_LABEL_DIR, exist_ok=True)

# --- Augmentation設定 ---
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.6),   # 明るさとコントラストのランダム調整
    A.HueSaturationValue(p=0.6),         # 色相・彩度・明度の変化
    A.Blur(blur_limit=3, p=0.3),         # ぼかし効果
    A.HorizontalFlip(p=0.5),             # 左右反転
    A.Affine(rotate=(-10, 10), scale=(0.9, 1.1), p=0.6),  # 回転とスケール変換
    A.Resize(640, 640)                   # YOLOでは画像サイズを統一する必要あり
],
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])
)

# --- YOLOラベルを読み込む関数 ---
def read_yolo_label(label_path):
    boxes = []
    class_labels = []
    if not os.path.exists(label_path):
        return boxes, class_labels
    with open(label_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls = int(parts[0])
            x, y, w, h = map(float, parts[1:])
            boxes.append([x, y, w, h])
            class_labels.append(cls)
    return boxes, class_labels

# --- YOLOラベルを書き出す関数 ---
def save_yolo_label(path, boxes, class_labels):
    with open(path, "w") as f:
        for cls, (x, y, w, h) in zip(class_labels, boxes):
            f.write(f"{int(cls)} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

# --- データ拡張の実行 ---
for img_name in tqdm(os.listdir(IMG_DIR)):
    if not img_name.endswith(".jpg"):
        continue

    img_path = os.path.join(IMG_DIR, img_name)
    label_path = os.path.join(LABEL_DIR, img_name.replace(".jpg", ".txt"))

    image = cv2.imread(img_path)
    bboxes, class_labels = read_yolo_label(label_path)

    # 各画像に対して複数のAugmentバージョンを作成
    for i in range(30):  # 各画像を30倍に増やす
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_img = transformed["image"]
        aug_bboxes = transformed["bboxes"]
        aug_labels = transformed["class_labels"]

        # バウンディングボックスがなくなった場合はスキップ
        if len(aug_bboxes) == 0:
            continue

        out_img_name = img_name.replace(".jpg", f"_aug_{i:03d}.jpg")
        out_label_name = img_name.replace(".jpg", f"_aug_{i:03d}.txt")

        cv2.imwrite(os.path.join(OUT_IMG_DIR, out_img_name), aug_img)
        save_yolo_label(os.path.join(OUT_LABEL_DIR, out_label_name), aug_bboxes, aug_labels)


100%|██████████| 18/18 [00:13<00:00,  1.38it/s]


In [ ]:
import cv2
import os

# 画像フォルダとラベルフォルダのパス
image_dir = "./data_train/v3/images/train"
label_dir = "./data_train/v3/labels/train"

# 画像ファイルの一覧を取得
for image_file in os.listdir(image_dir):
    if not image_file.endswith(".jpg"):
        continue

    img_path = os.path.join(image_dir, image_file)
    label_path = os.path.join(label_dir, image_file.replace(".jpg", ".txt"))

    img = cv2.imread(img_path)
    h, w, _ = img.shape

    # ラベルファイルが存在する場合、矩形を描画
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                cls_id = int(parts[0])
                x_center, y_center, box_w, box_h = map(float, parts[1:])

                # YOLO形式 → ピクセル座標に変換
                x1 = int((x_center - box_w / 2) * w)
                y1 = int((y_center - box_h / 2) * h)
                x2 = int((x_center + box_w / 2) * w)
                y2 = int((y_center + box_h / 2) * h)

                # 矩形を描画
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, str(cls_id), (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # 画像を表示
    cv2.imshow("Label Check", img)
    key = cv2.waitKey(0)
    if key == 27:  # ESCキーで終了
        break

cv2.destroyAllWindows()
